In [2]:
import pandas as pd
from datetime import datetime, timedelta
from calendar import monthrange
from collections import Counter

# Load the dataset
file_path = "aggregated_daily_data_with_date_aug2023_aug2024.csv"
data = pd.read_csv(file_path)

# Helper function to parse kg data into a dictionary for analysis
def parse_kg_data(kg_column, items_column):
    items = items_column.split(";")
    quantities = kg_column.split(";")
    return {item.strip(): int(qty) for item, qty in zip(items, quantities) if qty.isdigit()}

# Function to calculate the accurate date range for each week
def calculate_week_date_range(month_year, week):
    # Start date for the month
    start_date = datetime.strptime(month_year, "%b%Y")

    # Determine the start day for the week (e.g., week 1 starts at day 1)
    start_of_week = start_date + timedelta(days=(int(week) - 1) * 7)

    # Calculate the last day of the month
    last_day_of_month = monthrange(start_of_week.year, start_of_week.month)[1]
    end_of_month = start_of_week.replace(day=last_day_of_month)

    # Calculate end of the week
    end_of_week = start_of_week + timedelta(days=6)

    # Adjust if end_of_week exceeds the last day of the month
    if end_of_week > end_of_month:
        end_of_week = end_of_month

    return f"{start_of_week.strftime('%d/%m/%Y')}-{end_of_week.strftime('%d/%m/%Y')}"

# Function to generate weekly report with aggregated most/least consumed dishes in a single cell
def generate_weekly_report(data):
    weekly_summary = []

    # Loop through each row to accumulate consumption data for each week
    for _, row in data.iterrows():
        # Format the week identifier correctly without redundant "week" text
        week_key = f"{row['month_year']}_week{row['week'].replace('week', '')}"

        # Calculate the date range for the week
        date_range = calculate_week_date_range(row['month_year'], row['week'].replace('week', ''))

        # Parse the kg columns for each meal
        breakfast_data = parse_kg_data(row['breakfast_kg'], row['breakfast_items'])
        lunch_data = parse_kg_data(row['lunch_kg'], row['lunch_items'])
        dinner_data = parse_kg_data(row['dinner_kg'], row['dinner_items'])

        # Initialize week in summary if not already present
        if week_key not in [entry['Week'] for entry in weekly_summary]:
            # Aggregate for Breakfast
            most_consumed_breakfast = "; ".join([f"{item}: {qty}" for item, qty in Counter(breakfast_data).most_common(2)])
            least_consumed_breakfast = "; ".join([f"{item}: {qty}" for item, qty in Counter(breakfast_data).most_common()[:-3:-1]])

            # Aggregate for Lunch
            most_consumed_lunch = "; ".join([f"{item}: {qty}" for item, qty in Counter(lunch_data).most_common(2)])
            least_consumed_lunch = "; ".join([f"{item}: {qty}" for item, qty in Counter(lunch_data).most_common()[:-3:-1]])

            # Aggregate for Dinner
            most_consumed_dinner = "; ".join([f"{item}: {qty}" for item, qty in Counter(dinner_data).most_common(2)])
            least_consumed_dinner = "; ".join([f"{item}: {qty}" for item, qty in Counter(dinner_data).most_common()[:-3:-1]])

            # Append each meal's report to the weekly summary
            weekly_summary.append({
                'Week': week_key,
                'Date Range': date_range,
                'Meal': 'Breakfast',
                'Most Consumed Dishes (kg)': most_consumed_breakfast,
                'Least Consumed Dishes (kg)': least_consumed_breakfast
            })
            weekly_summary.append({
                'Week': week_key,
                'Date Range': date_range,
                'Meal': 'Lunch',
                'Most Consumed Dishes (kg)': most_consumed_lunch,
                'Least Consumed Dishes (kg)': least_consumed_lunch
            })
            weekly_summary.append({
                'Week': week_key,
                'Date Range': date_range,
                'Meal': 'Dinner',
                'Most Consumed Dishes (kg)': most_consumed_dinner,
                'Least Consumed Dishes (kg)': least_consumed_dinner
            })

    # Convert weekly summary to DataFrame and save as CSV
    weekly_summary_df = pd.DataFrame(weekly_summary)
    weekly_summary_df.to_csv("weekly_report.csv", index=False)

# Function to generate monthly report with aggregated most/least consumed dishes in a single cell
def generate_monthly_report(data):
    monthly_summary = []

    # Loop through each row to accumulate consumption data for each month
    for _, row in data.iterrows():
        month_year = row['month_year']

        # Parse the kg columns for each meal
        breakfast_data = parse_kg_data(row['breakfast_kg'], row['breakfast_items'])
        lunch_data = parse_kg_data(row['lunch_kg'], row['lunch_items'])
        dinner_data = parse_kg_data(row['dinner_kg'], row['dinner_items'])

        # Initialize month in summary if not already present
        if month_year not in [entry['Month'] for entry in monthly_summary]:
            # Aggregate for Breakfast
            most_consumed_breakfast = "; ".join([f"{item}: {qty}" for item, qty in Counter(breakfast_data).most_common(2)])
            least_consumed_breakfast = "; ".join([f"{item}: {qty}" for item, qty in Counter(breakfast_data).most_common()[:-3:-1]])

            # Aggregate for Lunch
            most_consumed_lunch = "; ".join([f"{item}: {qty}" for item, qty in Counter(lunch_data).most_common(2)])
            least_consumed_lunch = "; ".join([f"{item}: {qty}" for item, qty in Counter(lunch_data).most_common()[:-3:-1]])

            # Aggregate for Dinner
            most_consumed_dinner = "; ".join([f"{item}: {qty}" for item, qty in Counter(dinner_data).most_common(2)])
            least_consumed_dinner = "; ".join([f"{item}: {qty}" for item, qty in Counter(dinner_data).most_common()[:-3:-1]])

            # Append each meal's report to the monthly summary
            monthly_summary.append({
                'Month': month_year,
                'Meal': 'Breakfast',
                'Most Consumed Dishes (kg)': most_consumed_breakfast,
                'Least Consumed Dishes (kg)': least_consumed_breakfast
            })
            monthly_summary.append({
                'Month': month_year,
                'Meal': 'Lunch',
                'Most Consumed Dishes (kg)': most_consumed_lunch,
                'Least Consumed Dishes (kg)': least_consumed_lunch
            })
            monthly_summary.append({
                'Month': month_year,
                'Meal': 'Dinner',
                'Most Consumed Dishes (kg)': most_consumed_dinner,
                'Least Consumed Dishes (kg)': least_consumed_dinner
            })

    # Convert monthly summary to DataFrame and save as CSV
    monthly_summary_df = pd.DataFrame(monthly_summary)
    monthly_summary_df.to_csv("monthly_report.csv", index=False)

# Generate both reports
generate_weekly_report(data)
generate_monthly_report(data)
print("Weekly and Monthly reports saved as 'weekly_report.csv' and 'monthly_report.csv'.")


Weekly and Monthly reports saved as 'weekly_report.csv' and 'monthly_report.csv'.


In [4]:
def most_expanded_weekly(df):
  # Create an empty list to store the expanded rows
  most_expanded_data = []

  # Iterate over each row in the original dataset
  for index, row in df.iterrows():
      dishes = row['Most Consumed Dishes (kg)'].split(';')  # Split by semicolon to get each dish-quantity pair
      for dish in dishes:
          dish_name, quantity_kg = dish.split(':')  # Split each pair by colon
          most_expanded_data.append({
              'Week': row['Week'],
              'Date Range': row['Date Range'],
              'Meal': row['Meal'],
              'Dish Name': dish_name.strip(),
              'Quantity (kg)': float(quantity_kg.strip()) if quantity_kg.strip().replace('.', '', 1).isdigit() else None
          })

  # Convert the expanded data into a DataFrame
  most_expanded_df = pd.DataFrame(most_expanded_data)


  # Save the new dataset to a CSV
  most_expanded_df.to_csv('most_expanded_weekly_report.csv', index=False)

  print("Expanded weekly report for most consumed dishes saved as 'most_expanded_weekly_report.csv'.")



def least_expanded_weekly(df):
  # Create an empty list to store the expanded rows
  least_expanded_data = []

  # Iterate over each row in the original dataset
  for index, row in df.iterrows():
      dishes = row['Least Consumed Dishes (kg)'].split(';')  # Split by semicolon to get each dish-quantity pair
      for dish in dishes:
          dish_name, quantity_kg = dish.split(':')  # Split each pair by colon
          least_expanded_data.append({
              'Week': row['Week'],
              'Date Range': row['Date Range'],
              'Meal': row['Meal'],
              'Dish Name': dish_name.strip(),
              'Quantity (kg)': float(quantity_kg.strip()) if quantity_kg.strip().replace('.', '', 1).isdigit() else None
          })

  # Convert the expanded data into a DataFrame
  least_expanded_df = pd.DataFrame(least_expanded_data)


  # Save the new dataset to a CSV
  least_expanded_df.to_csv('least_expanded_weekly_report.csv', index=False)

  print("Expanded weekly report for least consumed dishes saved as 'least_expanded_weekly_report.csv'.")


def most_expanded_monthly(df):
  # Create an empty list to store the expanded rows
  most_expanded_data = []

  # Iterate over each row in the original dataset
  for index, row in df.iterrows():
      dishes = row['Most Consumed Dishes (kg)'].split(';')  # Split by semicolon to get each dish-quantity pair
      for dish in dishes:
          dish_name, quantity_kg = dish.split(':')  # Split each pair by colon
          most_expanded_data.append({
              'Month': row['Month'],
              'Meal': row['Meal'],
              'Dish Name': dish_name.strip(),
              'Quantity (kg)': float(quantity_kg.strip()) if quantity_kg.strip().replace('.', '', 1).isdigit() else None
          })

  # Convert the expanded data into a DataFrame
  most_expanded_df = pd.DataFrame(most_expanded_data)


  # Save the new dataset to a CSV
  most_expanded_df.to_csv('most_expanded_monthly_report.csv', index=False)

  print("Expanded monthly report for most consumed dishes saved as 'most_expanded_monthly_report.csv'.")


def least_expanded_monthly(df):
  # Create an empty list to store the expanded rows
  least_expanded_data = []

  # Iterate over each row in the original dataset
  for index, row in df.iterrows():
      dishes = row['Least Consumed Dishes (kg)'].split(';')  # Split by semicolon to get each dish-quantity pair
      for dish in dishes:
          dish_name, quantity_kg = dish.split(':')  # Split each pair by colon
          least_expanded_data.append({
              'Month': row['Month'],
              'Meal': row['Meal'],
              'Dish Name': dish_name.strip(),
              'Quantity (kg)': float(quantity_kg.strip()) if quantity_kg.strip().replace('.', '', 1).isdigit() else None
          })

  # Convert the expanded data into a DataFrame
  least_expanded_df = pd.DataFrame(least_expanded_data)


  # Save the new dataset to a CSV
  least_expanded_df.to_csv('least_expanded_monthly_report.csv', index=False)

  print("Expanded monthly report for least consumed dishes saved as 'least_expanded_monthly_report.csv'.")


# Load the data
df1 = pd.read_csv('weekly_report.csv')
df2 = pd.read_csv('monthly_report.csv')

#weekly expanded reports
most_expanded_weekly(df1)
least_expanded_weekly(df1)

#monthly expanded reports
most_expanded_monthly(df2)
least_expanded_monthly(df2)



Expanded weekly report for most consumed dishes saved as 'most_expanded_weekly_report.csv'.
Expanded weekly report for least consumed dishes saved as 'least_expanded_weekly_report.csv'.
Expanded monthly report for most consumed dishes saved as 'most_expanded_monthly_report.csv'.
Expanded monthly report for least consumed dishes saved as 'least_expanded_monthly_report.csv'.
